<a href="https://colab.research.google.com/github/adamdavidcole/stylegan3-fun-blend/blob/main/blend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Blending Experiment

test github connect

## Setup Google Drive Connection

In [ ]:
# Check GPU connection
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-5bc7d272-c87b-e7aa-cf20-1a95c4d78294)


In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
!pip install gdown --upgrade

if os.path.isdir("/content/drive/MyDrive/stylegan3-fun-blend"):
    %cd "/content/drive/MyDrive/stylegan3-fun-blend"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !git clone https://github.com/adamdavidcole/stylegan3-fun-blend.git
    %cd stylegan3-fun-blend
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    # !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/adamdavidcole/stylegan3-fun-blend.git
    %cd stylegan3-fun-blend
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/stylegan3-fun-blend


In [ ]:
# Update all code files in drive repo 
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- "*.py" 
!git checkout origin/main -- "*.ipynb"

Already up to date.
Saved working directory and index state WIP on main: 59cff72 drive+collab test3


In [ ]:
!pip install einops ninja gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 7.7 MB/s 


In [ ]:
import numpy as np
from datetime import datetime
import torch
import dnnlib
from dnnlib.util import format_time
import legacy
import PIL.Image

from torch_utils import gen_utils



## Network Setup

In [ ]:
network_key = "butterfly"
cfg="stylegan3-r"
pretrained_network = "ffhqu256"
dataset = "/content/drive/MyDrive/stylegan3/datasets/butterflys_256_2-256x256.zip"

results_outdir = f"./results/{network_key}"
training_outdir = f"{results_outdir}/training"

if not os.path.isdir(training_outdir):
    !mkdir -p $results_outdir

## Training

In [ ]:
!python train.py --outdir=$training_outdir --data=$dataset \
    --gpus=1 --batch=32 --batch-gpu=16  --gamma=6.6 --mirror=1 --kimg=50 --snap=1 --tick=1 --img-snap=1 --cbase=16384 \
    --cfg=$cfg --resume=$pretrained_network

Creating output directory...

Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2,
      "freeze_layers": 0,
      "freeze_embed": false
    },
    "channel_base": 32768,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9988915792636801,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
   

## Projection

In [ ]:
projection_source_images_outdir = "projection_source_images"
projection_source_vectors_outdir = "projection_source_vectors"


In [ ]:
# Upload File 
if not os.path.isdir(projection_source_images_outdir):
  !mkdir -p $projection_source_images_outdir

def upload_files():
  filepaths = []
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    filepath = f"{projection_source_images_outdir}/{k}"
    open(filepath, 'wb').write(v)
    filepaths.append(filepath)
  return list(filepaths)

uploaded = upload_files();
print(uploaded)

Saving alan3.jpg to alan3.jpg
['projection_source_images/alan3.jpg']


In [ ]:
# Project Image
# uploaded_file = uploaded[0]
uploaded_file="/content/drive/MyDrive/stylegan3-fun-blend/projection_source_images/alan3.jpg"
uploaded_file_name = uploaded_file.split('/')[-1].split('.')[0:-1]
uploaded_file_name = ''.join(uploaded_file_name)
print(uploaded_file)

!python projector.py --outdir=$projection_source_vectors_outdir --target=$uploaded_file --project-in-wplus --num-steps=5000 --save-video --stabilize-projection \
       --cfg=stylegan3-r --network=ffhqu256

/content/drive/MyDrive/stylegan3-fun-blend/projection_source_images/alan3.jpg
Loading networks from "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Projecting in W+ latent space...
Starting from W midpoint using 10000 samples...
Completed 0 of 5000
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
Completed 100 of 5000
Completed 200 of 5000
Completed 300 of 5000
Completed 400 of 5000
Completed 500 of 5000
Completed 600 of 5000
Completed 700 of 5000
Completed 800 of 5000
Completed 900 of 5000
Completed 1000 of 5000
Completed 1100 of 5000
Completed 1200 of 5000
Completed 1300 of 5000
Completed 1400 of 5000
Completed 1500 of 5000
Completed 1600 of 5000
Completed 1700 of 5000
Completed 1800 of 5000
Completed 1900 of 5000
Completed 2000 of 5000
Completed 2100 of 5000
Completed 2200 of 5000
Completed 2300 of 5000
Completed 2400 of 5000
Completed 2500 of 5000
Comple

## Projection Generation

In [ ]:
### Get set of most recent training checkpoints
most_recent_training_result = os.listdir(training_outdir)[-1]
print(most_recent_training_result)

path_to_most_recent_training_result = f"{training_outdir}/{most_recent_training_result}"
training_checkpoints = [f"{path_to_most_recent_training_result}/{f}" for f in os.listdir(path_to_most_recent_training_result) if f.endswith('.pkl')]
print(training_checkpoints)

./results/butterfly
00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256
['./results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000000.pkl', './results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000001.pkl', './results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000002.pkl', './results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000003.pkl', './results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000004.pkl', './results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000005.pkl', './results/butterfly/training/00008-stylegan3

In [ ]:
# projected_w_path = f"{projection_source_vectors_outdir}/00001-projection-w-wavgstart-sgan2/projected_wavg_final.npy"
# result_name="test_result"

ffhqu256_pkl = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"];
# projection_network_pkl = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"];
network_name="stylegan3-r_ffhqu256"

# projection_network_pkl = 

projection_outdir=f"{results_outdir}/projections"
# os.listdir(training_outdir)[-2]

projection_version_count = len(os.listdir(projection_outdir)) + 1
projection_version_count_padded = f'{projection_version_count:04}'
projection_outdir_version = f"{projection_outdir}/{projection_version_count_padded}_{network_name}"



print(projection_outdir_version)

if not os.path.isdir(projection_outdir_version):
  !mkdir -p $projection_outdir_version

./results/butterfly/projections/0008_stylegan3-r_ffhqu256


In [ ]:
# Generate image from projection
projected_w = np.load(projected_w_path)
print(projected_w.shape)

NameError: ignored

In [ ]:
# Generate N images across network blend
import torch
import dnnlib
from dnnlib.util import format_time
import legacy
import PIL.Image

from torch_utils import gen_utils


# projection_network_pkl = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"]


def gen_img_from_network(network_pkl_path):
  network_pkl_name = network_pkl_path.split('/')[-1]
  print('Loading networks from "%s"...' % network_pkl_path)
  device = torch.device('cuda')
  with dnnlib.util.open_url(network_pkl_path) as fp:
      G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

  projected_w_tensor = torch.tensor(projected_w).to(device)
  synth_image = gen_utils.w_to_img(G, dlatents=projected_w_tensor, noise_mode='const')[0]
  PIL.Image.fromarray(synth_image, 'RGB').save(f'{projection_outdir}/{network_pkl_name}.jpg')

for training_checkpoint in training_checkpoints:
  gen_img_from_network(training_checkpoint)

## Debugging Image Generation

My images dont look similar to the seeds so I'm investigating if there is a discrepency in my generation pipeline

In [ ]:
from torch_utils import gen_utils

generated_images_outdir = f"{results_outdir}/gen_images"
generated_images_network = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"];
network_name = "stylegan3-r_ffhqu256"

# generated_images_network = training_checkpoints[13];
# network_name = f"{network_key}_{generated_images_network.split('/')[-1]}"

In [ ]:
generated_images_version_count = len(os.listdir(generated_images_outdir))
generated_images_version_count_padded = f'{generated_images_version_count:04}'
generated_images_outdir_version = f"{generated_images_outdir}/{generated_images_version_count_padded}_{network_name}"
print(f"Outputting to {generated_images_outdir_version}")

!python gen_images.py --outdir=$generated_images_outdir_version --trunc=1 --seeds=0 --save_vectors=True \
        --network=$generated_images_network

Outputting to ./results/butterfly/gen_images/0005_stylegan3-r_ffhqu256
Loading networks from "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl"...
^C


In [ ]:
from torch_utils import gen_utils

def z_to_img(G, latents: torch.Tensor, label: torch.Tensor, truncation_psi: float, noise_mode: str = 'const') -> np.ndarray:
    """
    Get an image/np.ndarray from a latent Z using G, the label, truncation_psi, and noise_mode. The shape
    of the output image/np.ndarray will be [len(latents), G.img_resolution, G.img_resolution, G.img_channels]
    """
    dlatents = gen_utils.z_to_dlatent(G=G, latents=latents, label=label, truncation_psi=truncation_psi)
    dlatents = G.mapping.w_avg + (G.mapping.w_avg - dlatents) * truncation_psi
    img = gen_utils.w_to_img(G=G, dlatents=dlatents, noise_mode=noise_mode)  # Let's not redo code
    return img


In [ ]:
### Generate image from Z
from torch_utils import gen_utils

z_path = "/content/drive/MyDrive/stylegan3-fun-blend/results/butterfly/gen_images/0005_stylegan3-r_ffhqu256/0000_z.npy"

def gen_img_from_network_and_z(network_pkl_path, z_file_path):
  file_output_dir = f'{generated_images_outdir_version}/z_to_img2.jpg'

  network_pkl_name = network_pkl_path.split('/')[-1]
  print('Loading networks from "%s"...' % network_pkl_path)
  device = torch.device('cuda')
  with dnnlib.util.open_url(network_pkl_path) as fp:
      G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

  z_np = np.load(z_file_path)[0]
  z = torch.tensor(z_np).to(device)
  label = torch.zeros([1, G.c_dim], device=device)

  # These produce DIFFERENT results ?!?!?
  # synth_image = z_to_img(G, latents=z, label=label, truncation_psi=1)[0]
  # PIL.Image.fromarray(synth_image, 'RGB').save(file_output_dir)

  img = G(z, label, truncation_psi=1, noise_mode="const")
  img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
  PIL.Image.fromarray(img[0], 'RGB').save(file_output_dir)


  print(f"Outputted file to {file_output_dir}")
gen_img_from_network_and_z(generated_images_network, z_path)

Loading networks from "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl"...


KeyboardInterrupt: ignored

In [ ]:
### Generate from W
def gen_img_from_network_and_w(network_pkl_path, w_file_path):
  generated_image_count = len(os.listdir(projection_outdir_version))
  generated_image_count_padded = f"{generated_image_count:04}"
  file_output_dir = f'{projection_outdir_version}/{generated_image_count_padded}_w_to_img.jpg'
  network_pkl_name = network_pkl_path.split('/')[-1]
  print('Loading networks from "%s"...' % network_pkl_path)
  device = torch.device('cuda')
  with dnnlib.util.open_url(network_pkl_path) as fp:
      G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

  w_np = np.load(projected_w_path)[0]
  print(w_np.shape)
  w = torch.tensor(w_np).to(device)
  # print(w)

  # img = G.synthesis(w.unsqueeze(0), noise_mode="const")
  # img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  # img = PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{file_output_dir}')
  synth_image = gen_utils.w_to_img(G, dlatents=w, noise_mode='const')[0]
  PIL.Image.fromarray(synth_image, 'RGB').save(f'{file_output_dir}')
  print(f"Outputted image to {file_output_dir}")


In [ ]:
# projected_w_path = "/content/drive/MyDrive/stylegan3-fun-blend/projection_source_vectors/00004-projection-wplus-wavgstart-sgan2/projected_wplus_wavg_final.npy"
projected_w_path = "/content/drive/MyDrive/stylegan3-fun-blend/projection_source_vectors/00008-projection-wplus-wavgstart-sgan2/projected_wplus_wavg_final.npy"
# gen_img_from_network_and_w(training_checkpoints[13], projected_w_path)
# gen_img_from_network_and_w(generated_images_network, projected_w_path)

In [ ]:
for training_checkpoint in training_checkpoints:
  gen_img_from_network_and_w(training_checkpoint, projected_w_path)

Loading networks from "./results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000000.pkl"...
(16, 512)
Outputted image to ./results/butterfly/projections/0006_stylegan3-r_ffhqu256/0000_w_to_img.jpg
Loading networks from "./results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000001.pkl"...
(16, 512)
Outputted image to ./results/butterfly/projections/0006_stylegan3-r_ffhqu256/0001_w_to_img.jpg
Loading networks from "./results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000002.pkl"...
(16, 512)
Outputted image to ./results/butterfly/projections/0006_stylegan3-r_ffhqu256/0002_w_to_img.jpg
Loading networks from "./results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000003.pkl"...
(16, 512)
Outputted i

## Selection

## Refinement

## Blending Script

In [ ]:
blend_to_pkl = training_checkpoints[-1]

!python stylegan_blending.py --network1 $ffhqu256_pkl \
        --network2 $blend_to_pkl \
            --outdir butterflys/blend --dim 256

Loading networks from https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl and ./results/butterfly/training/00008-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000050.pkl ...
Generator(
  (synthesis): SynthesisNetwork(
    w_dim=512, num_ws=16,
    img_resolution=256, img_channels=3,
    num_layers=14, num_critical=2,
    margin_size=10, num_fp16_res=4
    (input): SynthesisInput(
      w_dim=512, channels=1024, size=[36, 36],
      sampling_rate=16, bandwidth=2
      (affine): FullyConnectedLayer(in_features=512, out_features=4, activation=linear)
    )
    (L0_36_1024): SynthesisLayer(
      w_dim=512, is_torgb=False,
      is_critically_sampled=False, use_fp16=False,
      in_sampling_rate=16, out_sampling_rate=16,
      in_cutoff=2, out_cutoff=2,
      in_half_width=6, out_half_width=6,
      in_size=[36, 36], out_size=[36, 36],
      in_channels=1024, out_channels=1024
      (af